# Examples Of Pyspark ML

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()


In [4]:
## Read The dataset
training = spark.read.csv('file:///home/genet/Desktop/Test1.csv',header=True,inferSchema=True)

In [5]:
training.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
| Genet| 31|        10| 30000|
| Ababe| 30|         8| 25000|
|Kebede| 29|         4| 20000|
| Sofil| 24|         3| 20000|
|Meryum| 21|         1| 15000|
|  Sani| 23|         2| 18000|
+------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [12]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [13]:
output=featureassembler.transform(training)



In [14]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
| Genet| 31|        10| 30000|         [31.0,10.0]|
| Ababe| 30|         8| 25000|          [30.0,8.0]|
|Kebede| 29|         4| 20000|          [29.0,4.0]|
| Sofil| 24|         3| 20000|          [24.0,3.0]|
|Meryum| 21|         1| 15000|          [21.0,1.0]|
|  Sani| 23|         2| 18000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [17]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [18]:
finalized_data=output.select("Independent Features","Salary")

In [19]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [21]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

23/07/25 14:03:00 WARN Instrumentation: [9441e1da] regParam is zero, which might cause numerical instability and overfitting.


In [24]:
### Coefficients
regressor.coefficients

DenseVector([2142.8571, -714.2857])

In [25]:
### Intercepts
regressor.intercept

-29285.71428567669

In [26]:
### Prediction
pred_results=regressor.evaluate(test_data)


In [27]:


pred_results.predictions.show()



+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000| 18571.42857142755|
|          [29.0,4.0]| 20000|29999.999999991876|
|          [30.0,8.0]| 25000|29285.714285712344|
+--------------------+------+------------------+

